### Python script to merge GPS information form AWI Polar-X planes
author Ingmar Nitze - ingmar.nitze@awi.de

#### ToDo List
None

In [ ]:
import pandas as pd
import os
import glob
import geopandas as gpd
import fiona
from shapely.geometry import Point, LineString, shape
fiona.drvsupport.supported_drivers['KML'] = 'rw' 

### Settings 

In [ ]:
#DIRECTORY = r'C:\Users\initze\OneDrive\136_Expedition_Planning\2021_PermaX_WestAlaska\GPS_Export\GPS1'
DIRECTORY = r'N:\Response\Restricted_Airborne\MACs\2021_Perma-X_Alaska\00_GPS\GPS_1'

### Functions 

In [ ]:
def load_file(f):
    cname = (os.path.basename(f).split('.')[1]).split('__')[0]
    df = pd.read_table(f, delim_whitespace=True, header=None, names=['timestamp', cname], index_col='timestamp')
    return df

In [ ]:
def to_decimal_degrees(number, factor=1):
    s = str(number)
    splitlen = len(s.split('.')[0])
    float(s[splitlen-2:])/60
    return float(s[:splitlen-2]) + float(s[splitlen-2:])/60

In [ ]:
def process_dataset(directory):
    
    # get filename
    filename = os.path.split(directory)[-1]
    
    # Create filelist and import to Dataframe
    lats = flist = glob.glob(directory + '/*/*.LAT___*.dat')
    lons = flist = glob.glob(directory + '/*/*.LON___*.dat')
    flist = lats + lons
    
    # load and merge data
    df_list = []
    for f in flist:
        try:
            df_tmp = load_file(f)
            df_list.append(df_tmp)
        except:
            continue
    df = pd.concat(df_list, axis=1)
    
    df_ll = df.dropna(subset=['LON']).drop_duplicates().reset_index().dropna(axis=1)
    df_ll['LON'] = df_ll['LON'].apply(to_decimal_degrees)*-1
    df_ll['LAT'] = df_ll['LAT'].apply(to_decimal_degrees)

    geom = gpd.points_from_xy(df_ll.LON, df_ll.LAT)
    
    # Create Geodataframe and export
    gdf = gpd.GeoDataFrame(df_ll, geometry=geom)

    outfile = f'{filename}.geojson'
    gdf.to_file(outfile, driver='GeoJSON')
    # Convert points to track/Line 
    outfile_line_json = f'{filename}_line.geojson'
    outfile_line_kml = f'{filename}_line.kml'

    gdf_line = LineString(gdf.geometry.tolist())
    gds = gpd.GeoSeries(data=gdf_line)

    if not os.path.exists(outfile_line_json):
        gds.to_file(outfile_line_json, driver='GeoJSON')
    else:
        print('File already exists')

    if not os.path.exists(outfile_line_kml):
        gds.to_file(outfile_line_kml, driver='KML')
    else:
        print('File already exists')

#### Check for input sub-directories 

In [ ]:
directories = [d for d in glob.glob(DIRECTORY + '/*') if os.path.isdir(d)]

In [ ]:
directories

#### Run processing 

In [ ]:
for d in directories:
    process_dataset(d)